In [1]:
import numpy as np
import pandas as pd



In [2]:
df = pd.read_json("Agenda_XIV.json")

In [3]:
df.head()

,reunions
reunion,"[{'cycleDeVie': {'etat': 'confirmé', 'chrono':..."


In [4]:
import json
json1_file = open('Agenda_XIV.json')
json1_str = json1_file.read()
json1_data = json.loads(json1_str)


In [5]:
first_meeting = json1_data['reunions']['reunion'][0]

In [6]:
first_meeting

{'@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
 'cycleDeVie': {'chrono': {'cloture': None,
   'creation': '2016-01-22T11:39:19.000+01:00'},
  'etat': 'confirmé'},
 'demandeurs': {'acteur': {'acteurRef': 'PA588872',
   'nom': 'Mme Cécile DUFLOT'}},
 'lieu': {'code': 'SALREU005',
  'libelleCourt': '2ème Bureau',
  'libelleLong': '2ème Bureau, Palais Bourbon'},
 'organeReuniRef': None,
 'timeStampDebut': '2016-03-15T10:00:00.000+01:00',
 'timeStampFin': '2016-03-15T12:00:00.000+01:00',
 'typeReunion': 'DEP',
 'uid': 'RUANR5L14S2016IDF0076295'}

In [7]:
print(json1_data['reunions']['reunion'][0]['demandeurs']['acteur']['acteurRef'])

PA588872


In [8]:
df2 = pd.DataFrame(json1_data['reunions']['reunion'])
df2.demandeurs.value_counts()

{'acteur': {'acteurRef': 'PA2338', 'nom': 'M. Christian PAUL'}}                                                                                                                                               597
{'acteur': {'acteurRef': 'PA1886', 'nom': 'M. Bruno LE ROUX'}}                                                                                                                                                589
{'acteur': {'acteurRef': 'PA610968', 'nom': 'M. Pouria AMIRSHAHI'}}                                                                                                                                           342
{'acteur': {'acteurRef': 'PA607553', 'nom': 'M. Dominique POTIER'}}                                                                                                                                           337
{'acteur': {'acteurRef': 'PA609764', 'nom': 'M. Yannick MOREAU'}}                                                                                               

In [81]:

def parse_ordre_du_jour(reu , df):
        lOrdreDuJour = reu.get('ODJ')
        # {'convocationODJ': None, 'pointsODJ': None, 'resumeODJ': None} 
        # {'convocationODJ': None, 'pointsODJ': None, 
        #  'resumeODJ': {'item': ['nomination du bureau ;', 'nomination des rapporteurs ;', 
        #                            'examen des dispositions restant en discussion du projet de loi.']}} 
        if(lOrdreDuJour):
            lConvoc = lOrdreDuJour.get('convocationODJ');
            if(lConvoc is not None):
               lConvoc = lConvoc.get('item');
            if(lConvoc):
                if(not isinstance(lConvoc, (list))):
                    lConvoc = [lConvoc];
                for i in range(len(lConvoc)):
                    df['ordre_du_jour_convoc_' + str(i+1)] = lConvoc[i];
            lPoints = lOrdreDuJour.get('pointsODJ');
            if lPoints is not None:
                lPoints = lPoints.get('item') 
            if(lPoints):
                if(not isinstance(lPoints, (list))):
                    lPoints = [lPoints];
                for i in range(len(lPoints)):
                    df['ordre_du_jour_point_' + str(i+1)] = lPoints[i];
            lResume = lOrdreDuJour.get('resumeODJ');
            if(lResume is not None):
                lResume = lResume.get('item');
            if(lResume):
                if(not isinstance(lResume, (list))):
                    lResume = [lResume];
                for i in range(len(lResume)):
                    df['ordre_du_jour_resume_' + str(i+1)] = lResume[i];
            

def parse_cycle_de_vie(reu , df):
        df['etat'] = None
        df['creation'] = None
        df['cloture'] = None
        lCycleDeVie = reu.get('cycleDeVie');
        if(lCycleDeVie):
            df['etat'] = lCycleDeVie['etat']
            df['creation'] = lCycleDeVie['chrono']['creation']
            df['cloture'] = lCycleDeVie['chrono']['cloture']
    
def parse_acteurs(reu , df):
        lDemandeurs = reu.get('demandeurs');
        if(lDemandeurs):
            lActeur = lDemandeurs.get('acteur');
            if(lActeur is not None):
                if(not isinstance(lActeur, (list))):
                    lActeur = [lActeur];
                for i in range(len(lActeur)):
                    df['acteur_ref_' + str(i+1)] = lActeur[i]['acteurRef']
                    df['acteur_nom_' + str(i+1)] = lActeur[i]['nom']
            lOrgane = lDemandeurs.get('organe');
            if(lOrgane is not None):
                if(not isinstance(lOrgane, (list))):
                    lOrgane = [lOrgane];
                for i in range(len(lOrgane)):
                    df['organe_ref_' + str(i+1)] = lOrgane[i]['organeRef']
                    df['organe_nom_' + str(i+1)] = lOrgane[i]['nom']

                    

def read_all(json_list):
    df0 = pd.DataFrame()
    for i in range(len(json_list)):
        if((i % 1000) == 0):
            print("READING" , i, "/" , len(json_list))
        reu = json_list[i]
        df1 = pd.DataFrame();
        df1['id'] = [reu['uid']]
        # print(reu['uid'])
        parse_ordre_du_jour(reu , df1);
        parse_cycle_de_vie(reu , df1);
        parse_acteurs(reu , df1);
        
        df1['lieu_code'] = reu['lieu'].get('code')
        df1['lieu_libelle_court'] = reu['lieu'].get('libelleCourt')
        df1['lieu_libelle_long'] = reu['lieu'].get('libelleLong')
        df1['organe'] = reu.get('organeReuniRef')
        df1['heure_debut'] = reu.get('timeStampDebut')
        df1['heure_fin'] = reu.get('timeStampFin')
        # print(df1.head())
        df0 = pd.concat([df0 , df1], axis = 0);
    return df0;



In [85]:
df3 = read_all(json1_data['reunions']['reunion'])

READING 0 / 37009
READING 1000 / 37009
READING 2000 / 37009
READING 3000 / 37009
READING 4000 / 37009
READING 5000 / 37009
READING 6000 / 37009
READING 7000 / 37009
READING 8000 / 37009
READING 9000 / 37009
READING 10000 / 37009
READING 11000 / 37009
READING 12000 / 37009
READING 13000 / 37009
READING 14000 / 37009
READING 15000 / 37009
READING 16000 / 37009
READING 17000 / 37009
READING 18000 / 37009
READING 19000 / 37009
READING 20000 / 37009
READING 21000 / 37009
READING 22000 / 37009
READING 23000 / 37009
READING 24000 / 37009
READING 25000 / 37009
READING 26000 / 37009
READING 27000 / 37009
READING 28000 / 37009
READING 29000 / 37009
READING 30000 / 37009
READING 31000 / 37009
READING 32000 / 37009
READING 33000 / 37009
READING 34000 / 37009
READING 35000 / 37009
READING 36000 / 37009
READING 37000 / 37009


In [86]:
df3.sample(12)

,acteur_nom_1,acteur_nom_2,acteur_nom_3,acteur_ref_1,acteur_ref_2,acteur_ref_3,cloture,creation,etat,heure_debut,...,ordre_du_jour_resume_3,ordre_du_jour_resume_4,ordre_du_jour_resume_5,ordre_du_jour_resume_6,ordre_du_jour_resume_7,ordre_du_jour_resume_8,ordre_du_jour_resume_9,organe,organe_nom_1,organe_ref_1
0,M. Pouria AMIRSHAHI,NaN,NaN,PA610968,NaN,NaN,None,2015-09-11T16:21:53.000+01:00,confirmé,2015-11-13T12:30:00.000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
0,M. Christian PAUL,NaN,NaN,PA2338,NaN,NaN,None,2015-07-15T17:33:35.000+02:00,Confirmé,2016-06-27T16:30:00.000+02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
0,M. Gwenegan BUI,NaN,NaN,PA642739,NaN,NaN,2015-12-08T15:08:37.000+01:00,2015-11-10T0016:29.000+01:00,annulé,2016-02-03T13:30:00.000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
0,M. Yves BLEIN,NaN,NaN,PA608641,NaN,NaN,None,2014-06-19T17:34:02.0Z,confirmé,2014-07-08T17:00:00.0Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
0,M. Patrick BLOCHE,NaN,NaN,PA543,NaN,NaN,None,2016-06-09T16:19:16.000+02:00,Confirmé,2016-06-15T10:00:00.000+02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
0,M. Bruno LE ROUX,NaN,NaN,PA1886,NaN,NaN,None,2015-01-09T20:13:32.000+01:00,confirmé,2015-01-21T14:00:00.000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,2016-02-18T00:00:00.000+01:00,2016-02-11T00:00:00.000+01:00,Supprimé,2016-03-01T17:00:00.000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PO59047,NaN,NaN
0,M. Alain CLAEYS,NaN,NaN,PA872,NaN,NaN,None,2014-05-21T14:02:02.0Z,confirmé,2014-05-27T17:30:00.0Z,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
0,M. Christian PAUL,NaN,NaN,PA2338,NaN,NaN,None,2015-05-21T14:49:53.000+02:00,confirmé,2015-07-21T16:00:00.000+02:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
0,M. Christian JACOB,NaN,NaN,PA1695,NaN,NaN,2015-02-20T16:26:37.000+01:00,2014-12-11T15:18:03.000+01:00,annulé,2015-02-24T09:30:00.000+01:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


In [87]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37009 entries, 0 to 0
Data columns (total 49 columns):
acteur_nom_1               25420 non-null object
acteur_nom_2               414 non-null object
acteur_nom_3               11 non-null object
acteur_ref_1               25420 non-null object
acteur_ref_2               414 non-null object
acteur_ref_3               11 non-null object
cloture                    6095 non-null object
creation                   36797 non-null object
etat                       36797 non-null object
heure_debut                37009 non-null object
heure_fin                  34304 non-null object
id                         37009 non-null object
lieu_code                  29915 non-null object
lieu_libelle_court         27650 non-null object
lieu_libelle_long          36407 non-null object
ordre_du_jour_convoc_1     7039 non-null object
ordre_du_jour_convoc_10    12 non-null object
ordre_du_jour_convoc_11    7 non-null object
ordre_du_jour_convoc_12    6 non

In [88]:
df3['acteur_nom_1'].value_counts().head(15) , df3['organe_nom_1'].value_counts().head(15) 

(M. Christian PAUL                 598
 M. Bruno LE ROUX                  590
 M. Pouria AMIRSHAHI               342
 M. Dominique POTIER               337
 M. Yannick MOREAU                 332
 M. Christophe SIRUGUE             315
 Mme Bernadette LACLAIS            287
 M. Philip CORDERY                 280
 M. Guillaume GAROT                276
 M. François PUPPONI               249
 Mme Seybah DAGOMA                 246
 M. Jean-François LAMOUR           245
 Mme Nathalie KOSCIUSKO-MORIZET    239
 M. Dominique LEFEBVRE             239
 M. Régis JUANICO                  229
 Name: acteur_nom_1, dtype: int64,
 GROUPE SOCIALISTE, RÉPUBLICAIN ET CITOYEN                 163
 GROUPE LES RÉPUBLICAINS                                   148
 GROUPE RADICAL, RÉPUBLICAIN, DÉMOCRATE ET PROGRESSISTE    137
 GROUPE UNION DES DÉMOCRATES ET INDÉPENDANTS               131
 GROUPE ÉCOLOGISTE                                         123
 GROUPE DE LA GAUCHE DÉMOCRATE ET RÉPUBLICAINE             122
 G

In [99]:
df3.lieu_libelle_long.value_counts()

Salle N° 3, 3, rue Aristide Briand                                                      1813
5ème Bureau, Palais Bourbon                                                             1731
Assemblée nationale                                                                     1626
4ème Bureau, Palais Bourbon                                                             1594
6ème Bureau, Palais Bourbon                                                             1574
2ème Bureau, Palais Bourbon                                                             1459
Salle N° 2, 3, rue Aristide Briand                                                      1279
Salon Gabriel, 101, rue de l'Université                                                 1252
Salon Visconti, 101, rue de l'Université                                                1198
Salle 6809, Palais Bourbon                                                              1089
Salle 7207, 101, rue de l'Université                                  

In [100]:
df3.to_csv("Agenda_XIV.csv")

In [98]:
for col in df3.columns:
    print("\n\nPROCESSING ******************** " , col);
    try:
        counts = df3[col].apply(str).value_counts();
        # print(counts[0:5])
        for i in range(5):
            print(str(counts.index[i])[0:50] , "\t", counts[i]);
    except:
        pass



PROCESSING ********************  acteur_nom_1
nan 	 11589
M. Christian PAUL 	 598
M. Bruno LE ROUX 	 590
M. Pouria AMIRSHAHI 	 342
M. Dominique POTIER 	 337


PROCESSING ********************  acteur_nom_2
nan 	 36595
M. Gilles SAVARY 	 49
M. Jean-Marie TÉTART 	 17
M. Hervé MARITON 	 17
M. Jean-Yves LE BOUILLONNEC 	 13


PROCESSING ********************  acteur_nom_3
nan 	 36998
M. Jérôme LAMBERT 	 2
M. Michel LIEBGOTT 	 1
M. Sébastien PIETRASANTA 	 1
M. Jacques PÉLISSARD 	 1


PROCESSING ********************  acteur_ref_1
nan 	 11589
PA2338 	 598
PA1886 	 590
PA610968 	 342
PA607553 	 337


PROCESSING ********************  acteur_ref_2
nan 	 36595
PA606495 	 49
PA609442 	 22
PA2083 	 17
PA2707 	 13


PROCESSING ********************  acteur_ref_3
nan 	 36998
PA610905 	 2
PA332523 	 1
PA607699 	 1
PA267585 	 1


PROCESSING ********************  cloture
None 	 30914
2015-01-05T00:00:00.000+01:00 	 25
2015-03-12T00:00:00.000+01:00 	 19
2014-04-03T00:00:00.000+02:00 	 18
2015-01-22T00:00:0